In [9]:
import pandas as pd
import numpy as np
import os

ANALYSIS ON IDS

In [10]:
#import IDS data
IDS_data = pd.read_csv(r'C:\Users\user\Desktop\MOPA Reconciliation\Reconciliation week of 28 Sept\Season Clients Detailed_20200928-093633.csv', usecols=['GovLocationGrandParent','GovLocationParent','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg'])


In [11]:
MOPA = pd.read_excel(r'C:\Users\user\Desktop\MOPA Reconciliation\Reconciliation week of 28 Sept\MOPA data.xlsx')
MOPA['SectorID']=MOPA['District']+'_'+MOPA['Sector']

In [12]:
#Preliminary info on IDS data on subsidized inputs
IDS_data['SectorID'] = IDS_data['GovLocationGrandParent']+'_'+IDS_data['GovLocationParent']
SectorID = MOPA['SectorID'].unique().tolist()
#GovLocationGrandParent = 'Bugesera'
#GovLocationParent = 'Shyara'
SectorDf = IDS_data [IDS_data ['SectorID'].isin(SectorID)]
SectorClients = SectorDf.groupby('SectorID')['OAFID'].count()
SectorDf = SectorDf.rename(columns={'GovLocationGrandParent':'GovDistrict', 'GovLocationParent':'Sector'})

SubsInputsdf = SectorDf[['SectorID','GovDistrict','Sector','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_NPK 17 kg','2021A_UREA kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg']]
SubsInputsdf['NationalID'] =SubsInputsdf['NationalID'].astype(str).replace('\.0', '', regex=True)

SubsInputsMelt = SubsInputsdf.melt(['SectorID','GovDistrict','Sector','SiteName','LastName','FirstName','OAFID','NationalID'],['2021A_DAP kg','2021A_NPK 17 kg','2021A_UREA kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg'], 'inputName','Quantity').fillna(0)

#Cleaning some IDS columns
SubsInputsMelt['NationalID'] =SubsInputsMelt['NationalID'].astype(str).replace('\.0', '', regex=True)

IDSInputsTxns = SubsInputsMelt[SubsInputsMelt['Quantity']>0]
IDSInputsTxns ['inputName'] = IDSInputsTxns ['inputName'].replace(['2021A_DAP kg','2021A_NPK 17 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_SC 403 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 637 kg','2021A_WH 505 kg','2021A_WH 605 kg'],['DAP','NPK','UREA','WH403','SC403','PAN53','PAN691','SC637','WH505','WH605'])
IDSInputsTxns =IDSInputsTxns.rename(columns={'inputName':'Product'})
IDSInputsTxns['ProductID'] = IDSInputsTxns['NationalID'] + IDSInputsTxns['Product']
Products = pd.DataFrame({'Product':['DAP', 'UREA', 'NPK', 'WH403','WH505','PAN53','SC403','SC637','WH605','PAN691']})
IDSInputsTxns = pd.merge(IDSInputsTxns,Products, on='Product', how='outer').fillna(0)

#Check if NID starts with 119
SubsInputsdf['NIDStartsWith119'] = SubsInputsdf['NationalID'].str.startswith('119')
NIDStartsWith119 = SubsInputsdf[SubsInputsdf['NIDStartsWith119']==False]
NIDStartsWith119_sector = NIDStartsWith119.groupby('SectorID')['OAFID'].count().fillna(0).reset_index()

#Check if NID contains either 700 for female and 800 for male
SubsInputsdf['FemaleCheck'] = SubsInputsdf['NationalID'].str.contains(pat = '700')
SubsInputsdf['maleCheck'] = SubsInputsdf['NationalID'].str.contains(pat = '800')
SubsInputsdf['FalseGender'] = SubsInputsdf['FemaleCheck']== SubsInputsdf['maleCheck']
FalseGender = SubsInputsdf[SubsInputsdf['FalseGender']==True]
FalseGender_sector = FalseGender.groupby('SectorID')['OAFID'].count().fillna(0).reset_index()

#Check if NID length is equal to 16
SubsInputsdf['NIDLength'] = SubsInputsdf['NationalID'].str.len()==16
NIDLength = SubsInputsdf[SubsInputsdf['NIDLength']==False]
NIDLength_sector = NIDLength.groupby('SectorID')['OAFID'].count().fillna(0).reset_index()

#FINAL NID FRAME
x = pd.merge(NIDStartsWith119_sector,FalseGender_sector, on='SectorID', how='outer').reset_index()
CheckOnNID = pd.merge(x,NIDLength_sector, on='SectorID',how='outer').fillna(0).reset_index()
CheckOnNID['#Wrong NID'] = CheckOnNID['OAFID_x']+CheckOnNID['OAFID_y']+CheckOnNID['OAFID']

#IDSClientsPerInput = IDSInputsTxns.groupby('Product')['OAFID'].count().reset_index()
#IDSQtyPerInput = IDSInputsTxns.groupby('Product')['Quantity'].sum().reset_index()
#SectorSummary = pd.merge(IDSClientsPerInput,IDSQtyPerInput,on='Product',how='outer').rename(columns={'OAFID':'#Clients'})
#SectorSummary 

#(1) FINDING THE TOTAL NUMBER OF CLIENTS IN EACH SECTOR ON IDS
TotalSectorClients = SectorDf.groupby('SectorID')['OAFID'].count().reset_index()

#(2)FINDING THE TOTAL NUMBER OF WRONG NID
ClientsWithWrongNID = CheckOnNID[['SectorID','#Wrong NID']]

#(3)FINDING THE TOTAL QUANTITY ON IDS IN EACH SECTOR
TotalKgsPerSector = pd.pivot_table(IDSInputsTxns, values='Quantity', index=['SectorID'],columns=['Product'], aggfunc='sum').reset_index().rename(columns={'DAP':'DAP I_kgs', 'NPK':'NPK I_kgs', 'SC403':'SC403 I_kgs' , 'UREA':'UREA I_kgs','WH403':'WH403 I_kgs','PAN53':'PAN53 I_kgs','PAN691':'PAN691 I_kgs','SC637':'SC637 I_kgs','WH505':'WH505 I_kgs','WH605':'WH605 I_kgs' })

#(4)TOTAL NUMBER OF CLIENTS WHO TOOK SPECIFIC PRODUCTS
ClientsPerInputs = pd.pivot_table(IDSInputsTxns, values='OAFID', index=['SectorID'],columns=['Product'], aggfunc='count').reset_index().rename(columns={'DAP':'DAP I#', 'NPK':'NPK I#', 'SC403':'SC403 I#' , 'UREA':'UREA I#','WH403':'WH403 I#','PAN53':'PAN53 I#','PAN691':'PAN691 I#','SC637':'SC637 I#','WH505':'WH505 I#','WH605':'WH605 I#'})

#(5)MATCHING THE NUMBER OF CLIENTS AND TOTAL KGS TAKEN
IDS = pd.merge(pd.merge(ClientsPerInputs,TotalKgsPerSector, on='SectorID', how='outer' ),ClientsWithWrongNID, on='SectorID', how='outer').fillna(0).drop([0])

ANALYSIS ON MOPA

In [13]:
#Cleaning MOPA data
#ColSplit = MOPA ['Requested Qty'].str.split(" ", n = 2, expand = True)
#MOPA['Requested_qty'] = ColSplit[0].astype(float)
MOPA['Paid_qty'] = MOPA['Paid Qty'].str.extract('(\d+)').astype(float)
MOPA['Product'] = MOPA['Product'].str.upper()
MOPA = MOPA.rename(columns={'National Id':'NationalID'})
MOPA['NationalID'] = MOPA['NationalID'].astype(str)
MOPA['ProductID'] = MOPA['NationalID'] + MOPA['Product']
MOPA['Product'] = MOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
MOPA_dataCols1 = MOPA[['District','SectorID','Name','Surname','NationalID','Product','Paid_qty']]
MOPA_dataCols = pd.merge(MOPA_dataCols1,Products,on='Product',how='outer').fillna(0)


In [14]:
MOPATotalKgsPerSector = pd.pivot_table(MOPA_dataCols, values='Paid_qty', index=['SectorID'],columns=['Product'], aggfunc='sum').reset_index().rename(columns={'DAP':'DAP M_kgs', 'NPK':'NPK M_kgs', 'SC403':'SC403 M_kgs' , 'UREA':'UREA M_kgs','WH403':'WH403 M_kgs','PAN691':'PAN691 M_kgs','SC637':'SC637 M_kgs','WH505':'WH505 M_kgs','PAN53':'PAN53 M_kgs','SC637':'SC637 M_kgs','WH605':'WH605 M_kgs'}).fillna(0)

MOPAClientsPerInputs = pd.pivot_table(MOPA_dataCols, values='NationalID', index=['SectorID'],columns=['Product'], aggfunc='count').reset_index().rename(columns={'DAP':'DAP M#', 'NPK':'NPK M#', 'SC403':'SC403 M#' , 'UREA':'UREA M#','WH403':'WH403 M#','PAN691':'PAN691 M#','SC637':'SC637 M#','WH505':'WH505 M#','PAN53':'PAN53 M#','SC637':'SC637 M#','WH605':'WH605 M#'}).fillna(0)

MOPA_data = pd.merge(MOPATotalKgsPerSector,MOPAClientsPerInputs, on='SectorID', how='outer' ).drop([0])


In [16]:
#COMBINING IDS AND MOPA DATA
IDSvsMOPA = pd.merge(IDS,MOPA_data, on='SectorID',how='outer').fillna(0)
IDSvsMOPA['Diff in DAP#'] = IDSvsMOPA['DAP I#']-IDSvsMOPA['DAP M#']
IDSvsMOPA['Diff in NPK#'] = IDSvsMOPA['NPK I#']-IDSvsMOPA['NPK M#']
IDSvsMOPA['Diff in UREA#'] = IDSvsMOPA['UREA I#']-IDSvsMOPA['UREA M#']
IDSvsMOPA['Diff in SC403#'] = IDSvsMOPA['SC403 I#']-IDSvsMOPA['SC403 M#']
IDSvsMOPA['Diff in SC637#'] = IDSvsMOPA['SC637 I#']-IDSvsMOPA['SC637 M#']
IDSvsMOPA['Diff in PAN53#'] = IDSvsMOPA['PAN53 I#'] - IDSvsMOPA['PAN53 M#']
IDSvsMOPA['Diff in PAN691#'] = IDSvsMOPA['PAN691 I#'] - IDSvsMOPA['PAN691 M#'] 
IDSvsMOPA['Diff in WH403#'] = IDSvsMOPA['WH403 I#']-IDSvsMOPA['WH403 M#']
IDSvsMOPA['Diff in WH505#'] = IDSvsMOPA['WH505 I#']-IDSvsMOPA['WH505 M#']
IDSvsMOPA['Diff in WH605#'] = IDSvsMOPA['WH605 I#']-IDSvsMOPA['WH605 M#']

IDSvsMOPA['Diff in DAP Kgs'] = IDSvsMOPA['DAP I_kgs']-IDSvsMOPA['DAP M_kgs']
IDSvsMOPA['Diff in NPK Kgs'] = IDSvsMOPA['NPK I_kgs']-IDSvsMOPA['NPK M_kgs']
IDSvsMOPA['Diff in UREA Kgs'] = IDSvsMOPA['UREA I_kgs']-IDSvsMOPA['UREA M_kgs']
IDSvsMOPA['Diff in SC403 Kgs'] = IDSvsMOPA['SC403 I_kgs']-IDSvsMOPA['SC403 M_kgs']
IDSvsMOPA['Diff in SC637 Kgs'] = IDSvsMOPA['SC637 I_kgs']-IDSvsMOPA['SC637 M_kgs']
IDSvsMOPA['Diff in PAN53 Kgs'] = IDSvsMOPA['PAN53 I_kgs'] - IDSvsMOPA['PAN53 M_kgs']
IDSvsMOPA['Diff in PAN691 Kgs'] = IDSvsMOPA['PAN691 I_kgs'] - IDSvsMOPA['PAN691 M_kgs'] 
IDSvsMOPA['Diff in WH505 Kgs'] = IDSvsMOPA['WH505 I_kgs']-IDSvsMOPA['WH505 M_kgs']
IDSvsMOPA['Diff in WH403 Kgs'] = IDSvsMOPA['WH403 I_kgs']-IDSvsMOPA['WH403 M_kgs']
IDSvsMOPA['Diff in WH605 Kgs'] = IDSvsMOPA['WH605 I_kgs']-IDSvsMOPA['WH605 M_kgs']

os.chdir(r'C:\Users\user\Desktop\MOPA Reconciliation\Reconciliation week of 28 Sept')

IDSvsMOPA.to_excel('Reconciliation1.xlsx')